In [1]:
!unzip data.zip

Archive:  data.zip
  inflating: Семинар 2/segmentation_exp_seria.py  
  inflating: Семинар 2/core/__pycache__/losses.cpython-310.pyc  
  inflating: Семинар 2/Руководство.docx  
  inflating: Семинар 2/core/trainers.py  
  inflating: Семинар 2/train_segmentator.ipynb  
  inflating: Семинар 2/core/dataset.py  
  inflating: Семинар 2/Test model .ipynb  
  inflating: Семинар 2/core/__pycache__/trainers.cpython-37.pyc  
  inflating: Семинар 2/runs/Combo_loss_clean_NN/IoU_Патология_train/events.out.tfevents.1744701380.ailab7525g-09.310960.3  
  inflating: Семинар 2/runs/Combo_loss_clean_NN/IoU_Патология_val/events.out.tfevents.1744701380.ailab7525g-09.310960.4  
  inflating: Семинар 2/runs/Combo_loss_all_weight_NN/IoU_Патология_val/events.out.tfevents.1744707780.ailab7525g-09.310960.9  
  inflating: Семинар 2/runs/BCE_clean_N/IoU_Патология_val/events.out.tfevents.1744656674.ailab7525g-09.3958416.4  
  inflating: Семинар 2/runs/BCE_pixelweight_N/IoU_Патология_train/events.out.tfevents.17446626

In [2]:
!pip install numpy matplotlib opencv-python torch torchvision pycocotools tensorflow tensorboard tqdm segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
import os
import sys
import json
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image
from torch.utils.data import random_split, DataLoader, ConcatDataset
import torch.nn as nn
from torchvision.models import efficientnet_b0

# ─────────────────────────────────────────────────────────────────────────────
# 1. Повторякем подготовку данных
# ─────────────────────────────────────────────────────────────────────────────

seminar_path = "/content/task"
seminar_path = Path(seminar_path)
data_path = f"{seminar_path}/data"
sys.path.insert(0, f"{seminar_path}/core")
print(sys.path)

from dataset import SimpleCocoDataset
from trainers import SimpleClassificationTrainer

pathology_ids = [i for i in range(6, 27) if i != 15]   # 6‑26, кроме 15

out_classes = [ {"id": 1, "name": "Патология", "summable_masks": pathology_ids, "subtractive_masks": []}]

base_names = [
    "Правое лёгкое", "Левое лёгкое", "Контуры сердца", "Купола диафрагмы и нижележащая область",
    "Сложный случай", "нельзя составить заключение", "Иная патология", "Гидроторакс",
    "Легочно-венозная гипертензия 2 стадии и выше", "Пневмоторакс", "Доброкачественное новообразование",
    "Перелом ребра свежий", "Буллезное вздутие, тонкостенная киста", "Рак лёгкого (включая дорожку к корню при наличии)",
    "Кардиомегалия (отмечается всё сердце, как патология)", "Интерстициальная пневмония.",
    "Метастатическое поражение лёгких", "Полость с уровнем жидкости", "Грыжа пищевого отверстия диафрагмы",
    "Спавшийся сегмент лёгкого при ателектазе", "Инфильтративный туберкулёз",
    "Пневмония. В том числе сегментарная и полисегментарная", "Область распада, деструкции тканей лёгкого",
    "Участок пневмофиброза", "Кальцинаты. Каждый кальцинат выделяется отдельным контуром",
    "Консолидированный перелом ребра"
]

base_classes = [{"id": i+1, "name": name} for i, name in enumerate(base_names)]


#Настроим параметры даталодера
batch_size = 64
resize = (512, 512)


data_roots   = {p.parent.parent for p in (seminar_path / "data").rglob("annotations/instances_default.json")}

print(f"Найдено {len(data_roots)} датасетов:", *data_roots, sep="\n  ")


datasets = [SimpleCocoDataset(str(d),
                              base_classes,
                              out_classes,
                              resize=resize)
            for d in sorted(data_roots)]

full_ds  = ConcatDataset(datasets)


['/content/task/core', '/home/alexskv/seminar_2/core', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmp19bztkhl']
Найдено 4 датасетов:
  /content/task/data/task4
  /content/task/data/task1
  /content/task/data/task2
  /content/task/data/task3
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [6]:
# ─────────────────────────────────────────────────────────────────────────────
# 2. Разделение на train/val
# ─────────────────────────────────────────────────────────────────────────────
val_percent = 0.2
val_size = int(len(full_ds) * val_percent)
train_size = len(full_ds) - val_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

val_loader = DataLoader(val_ds,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4)





/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# 3. Создаем модель EfficientNet адаптированная под 1 выход
# ─────────────────────────────────────────────────────────────────────────────
class MonoEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        model = efficientnet_b0(weights=None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
        self.backbone = model

    def forward(self, x):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)  # 1 канал → 3
        return self.backbone(x)

model = MonoEfficientNet()

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 4. Запуск обучения
# ─────────────────────────────────────────────────────────────────────────────
device = torch.device("cpu")

trainer = SimpleClassificationTrainer(
    model=model,
    classes = out_classes,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=10,
    exp_name="efficientnet_binary_cls"
)

trainer.train()

TRAIN:   0%|          | 0/16 [00:00<?, ?it/s]